In [ ]:
import cv2
import numpy as np
import open3d as o3d
import pickle
import time

from FR3Py.vision.utils import depth2PointCloud

In [ ]:
with open('dataset/sep26/camera_params.pkl', 'rb') as f:
    params = pickle.load(f)

int_params = params['int_params']
ext_params = params['ext_params']
fx, fy, cx, cy = int_params['Depth']['fx'], int_params['Depth']['fy'], int_params['Depth']['cx'], int_params['Depth']['cy']

In [ ]:

depth1 = cv2.imread('dataset/sep26/seq1/depth/1695784268680308480.png', -1)
depth2 = cv2.imread('dataset/sep26/seq1/depth/1695784281219218432.png', -1)

pc1 = depth2PointCloud(depth1,fx, fy, cx, cy).copy()
pc2 = depth2PointCloud(depth2,fx, fy, cx, cy).copy()

In [ ]:
pc1 = np.hstack([pc1, np.ones((pc1.shape[0], 1))])
pc2 = np.hstack([pc2, np.ones((pc2.shape[0], 1))])

base_T1_ef = np.loadtxt('dataset/sep26/seq1/pose/1695784268680308480.txt')
base_T2_ef = np.loadtxt('dataset/sep26/seq1/pose/1695784281219218432.txt')

rgb_T1_tag = np.loadtxt('dataset/sep26/seq1/apriltag_pose/1695784268680308480.txt')
rgb_T2_tag = np.loadtxt('dataset/sep26/seq1/apriltag_pose/1695784281219218432.txt')

tag_T1_rgb = np.linalg.inv(rgb_T1_tag)
tag_T2_rgb = np.linalg.inv(rgb_T2_tag)

depth_T_rgb = ext_params['ir1_T_rgb']
rgb_T_depth = np.linalg.inv(depth_T_rgb)

In [ ]:
tag_pc1 = ((tag_T1_rgb@rgb_T_depth@pc1.T).T)[:,0:3]
tag_pc2 = ((tag_T2_rgb@rgb_T_depth@pc2.T).T)[:,0:3]

In [ ]:
pcd1 = o3d.geometry.PointCloud()
pcd1.points = o3d.utility.Vector3dVector(pc1[:,0:3])
tag_pcd1 = o3d.geometry.PointCloud()
tag_pcd1.points = o3d.utility.Vector3dVector(tag_pc1[:,0:3])

pcd2 = o3d.geometry.PointCloud()
pcd2.points = o3d.utility.Vector3dVector(pc2[:,0:3])
tag_pcd2 = o3d.geometry.PointCloud()
tag_pcd2.points = o3d.utility.Vector3dVector(tag_pc2[:,0:3])

In [ ]:
o3d.visualization.draw_geometries([pcd1,pcd2])

In [ ]:
o3d.visualization.draw_geometries([tag_pcd1, tag_pcd2])

# Camera Feed

In [ ]:
from FR3Py.vision.cameras import RealSenseCamera
camera = RealSenseCamera(VGA=False, enable_imu=False, enable_ir=True, emitter_enabled=True, align_to_color=False)

In [ ]:
K = camera.getIntrinsics()['Depth']
fx,fy,cx,cy = K['fx'], K['fy'], K['cx'], K['cy']

## Snapshot

In [ ]:
camera.grab_frames()
depth = camera.depth_frame
pc = depth2PointCloud(depth,fx, fy, cx, cy)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pc)
o3d.visualization.draw_geometries([pcd])

## Live

In [ ]:
camera.grab_frames()
depth = camera.depth_frame
pc = depth2PointCloud(depth,fx, fy, cx, cy)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pc)
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(pcd)

while True:
    time.sleep(0.01)
    camera.grab_frames()
    depth = camera.depth_frame
    pc = depth2PointCloud(depth,fx, fy, cx, cy)
    pcd.points = o3d.utility.Vector3dVector(pc)
    vis.update_geometry(pcd)
    vis.poll_events()
    vis.update_renderer()